In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, VGG16, ResNet50
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_pre
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as res_pre
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input, BatchNormalization, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K
import gc
import math
from datetime import datetime

# -------------------------
# User params (modify as needed)
# -------------------------
BASE_PATH = '/kaggle/input/mango7200-12/MangoMerged'  
IMG_SIZE = (224, 224)
BATCH_SIZE = 32                 
NUM_FOLDS = 5
EPOCHS = 10
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
PDF_REPORT = "kfold_full_report.pdf"

# -------------------------
# Reproducibility
# -------------------------
np.random.seed(SEED)
tf.random.set_seed(SEED)

# -------------------------
# TPU / Strategy detection
# -------------------------
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("Running on TPU:", resolver.cluster_spec().as_dict())
except Exception as e:
    # Fallback to MirroredStrategy (multi-GPU) if available, else default strategy
    try:
        gpus = tf.config.experimental.list_logical_devices('GPU')
        if gpus:
            strategy = tf.distribute.MirroredStrategy()
            print("Running on MirroredStrategy with GPUs:", gpus)
        else:
            strategy = tf.distribute.get_strategy()
            print("Running on default strategy (CPU).")
    except:
        strategy = tf.distribute.get_strategy()
        print("Running on default strategy (CPU).")
print("Number of replicas:", strategy.num_replicas_in_sync)

# Scale batch size for distributed strategy (TPU/GPU)
GLOBAL_BATCH_SIZE = BATCH_SIZE * max(1, strategy.num_replicas_in_sync)

# -------------------------
# Helper: collect filepaths and one-hot labels
# -------------------------
def collect_filepaths_and_labels(base_dir):
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    class_to_index = {c: i for i, c in enumerate(class_names)}
    filepaths = []
    labels = []
    for cls in class_names:
        cls_dir = os.path.join(base_dir, cls)
        for fname in os.listdir(cls_dir):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                filepaths.append(os.path.join(cls_dir, fname))
                labels.append(class_to_index[cls])
    filepaths = np.array(filepaths)
    labels = np.array(labels, dtype=np.int32)
    return filepaths, labels, class_names

# -------------------------
# Build dataset mapping functions
# -------------------------
def decode_and_resize(path, label):
    # path: string scalar tensor of filepath
    image = tf.io.read_file(path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)
    image.set_shape([None, None, 3])
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)  # leave in 0-255 float; model lambdas will preprocess
    return image, label

def one_hot_label(image, label, num_classes):
    label = tf.one_hot(label, depth=num_classes)
    return image, label

def prepare_dataset(filepaths, labels, num_classes, is_training=True, batch_size=GLOBAL_BATCH_SIZE):
    ds = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    if is_training:
        ds = ds.shuffle(buffer_size=len(filepaths), seed=SEED)
    ds = ds.map(decode_and_resize, num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x, y: one_hot_label(x, y, num_classes), num_parallel_calls=AUTOTUNE)
    if is_training:
        ds = ds.repeat()  # we'll pass steps_per_epoch to fit
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    return ds

# -------------------------
# Model builder (inside strategy scope)
# -------------------------
def build_model(num_classes):
    input_shape = IMG_SIZE + (3,)
    inputs = Input(shape=input_shape, dtype=tf.float32)

    # Preprocess for each model separately (Lambda layers)
    eff_input = Lambda(lambda x: eff_pre(x), name="eff_pre")(inputs)
    vgg_input = Lambda(lambda x: vgg_pre(x), name="vgg_pre")(inputs)
    res_input = Lambda(lambda x: res_pre(x), name="res_pre")(inputs)

    # Base models
    effnet_base = EfficientNetB0(weights="imagenet", include_top=False)
    vgg16_base = VGG16(weights="imagenet", include_top=False)
    resnet_base = ResNet50(weights="imagenet", include_top=False)

    # Freeze base models for transfer learning
    for base in (effnet_base, vgg16_base, resnet_base):
        base.trainable = False

    # Extract features
    eff_features = effnet_base(eff_input)
    vgg_features = vgg16_base(vgg_input)
    res_features = resnet_base(res_input)

    feat1 = GlobalAveragePooling2D()(eff_features)
    feat2 = GlobalAveragePooling2D()(vgg_features)
    feat3 = GlobalAveragePooling2D()(res_features)

    merged = Concatenate()([feat1, feat2, feat3])

    x = Dense(512, activation="relu")(merged)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = Dense(256, activation="relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

# -------------------------
# Main K-Fold flow
# -------------------------
def run_kfold(base_path, num_folds=NUM_FOLDS, epochs=EPOCHS):
    # collect train + val together as "X_all"
    train_dir = os.path.join(base_path, "train")
    val_dir = os.path.join(base_path, "val")
    test_dir = os.path.join(base_path, "test")

    # Collect filepaths & labels for train+val
    train_files, train_labels, class_names_train = collect_filepaths_and_labels(train_dir)
    val_files, val_labels, class_names_val = collect_filepaths_and_labels(val_dir)

    # sanity: class names should match
    if class_names_train != class_names_val:
        print("Warning: train and val class order differ. Proceeding with union & sorted classes.")
    # Use sorted union of class names (to ensure consistent indexing)
    all_classes = sorted(list(set(class_names_train) | set(class_names_val)))
    class_to_idx = {c: i for i, c in enumerate(all_classes)}
    num_classes = len(all_classes)
    print("Detected classes:", all_classes)

    # Re-map train/val labels to this unified index
    def remap(filepaths, labels, original_class_names):
        remapped = []
        for p, lbl in zip(filepaths, labels):
            # derive class name from parent folder
            cls = os.path.basename(os.path.dirname(p))
            remapped.append(class_to_idx[cls])
        return np.array(remapped, dtype=np.int32)

    train_labels_m = remap(train_files, train_labels, class_names_train)
    val_labels_m = remap(val_files, val_labels, class_names_val)

    X_all_files = np.concatenate([train_files, val_files], axis=0)
    y_all_labels = np.concatenate([train_labels_m, val_labels_m], axis=0)

    # Test set
    test_files, test_labels, class_names_test = collect_filepaths_and_labels(test_dir)
    # Remap test labels to unified classes
    test_labels_m = []
    for p, lbl in zip(test_files, test_labels):
        cls = os.path.basename(os.path.dirname(p))
        test_labels_m.append(class_to_idx.get(cls, -1))
    test_labels_m = np.array(test_labels_m, dtype=np.int32)
    # filter out any test items not in class mapping (shouldn't happen)
    valid_mask = test_labels_m >= 0
    test_files = test_files[valid_mask]
    test_labels_m = test_labels_m[valid_mask]

    # K-Fold on indices
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=SEED)

    fold_train_acc, fold_train_loss = [], []
    fold_val_acc, fold_val_loss = [], []
    fold_test_acc, fold_test_loss = [], []
    fold_class_reports = []

    # PDF for all outputs
    with PdfPages(PDF_REPORT) as pdf:
        fold_no = 1
        for train_idx, val_idx in kf.split(X_all_files):
            print(f"\n===== Fold {fold_no} =====")
            K.clear_session()
            gc.collect()

            # Slice filepaths and labels for this fold
            train_files_fold = X_all_files[train_idx]
            train_labels_fold = y_all_labels[train_idx]
            val_files_fold = X_all_files[val_idx]
            val_labels_fold = y_all_labels[val_idx]

            # Build tf.data datasets (note: train dataset .repeat() is handled inside prepare_dataset)
            train_ds = prepare_dataset(train_files_fold, train_labels_fold, num_classes,
                                       is_training=True, batch_size=GLOBAL_BATCH_SIZE)
            val_ds = prepare_dataset(val_files_fold, val_labels_fold, num_classes,
                                     is_training=False, batch_size=GLOBAL_BATCH_SIZE)
            test_ds = prepare_dataset(test_files, test_labels_m, num_classes,
                                      is_training=False, batch_size=GLOBAL_BATCH_SIZE)

            # steps per epoch (since train_ds repeats)
            steps_per_epoch = math.ceil(len(train_files_fold) / GLOBAL_BATCH_SIZE)
            validation_steps = math.ceil(len(val_files_fold) / GLOBAL_BATCH_SIZE)
            test_steps = math.ceil(len(test_files) / GLOBAL_BATCH_SIZE)

            # Build model inside strategy scope
            with strategy.scope():
                model = build_model(num_classes)

            # Callbacks
            fold_ckpt = f"best_model_fold{fold_no}.h5"
            callbacks = [
                ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, verbose=1),
                EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True, verbose=1),
                ModelCheckpoint(fold_ckpt, monitor="val_loss", save_best_only=True, verbose=1)
            ]

            # Fit
            history = model.fit(train_ds,
                                epochs=epochs,
                                steps_per_epoch=steps_per_epoch,
                                validation_data=val_ds,
                                validation_steps=validation_steps,
                                callbacks=callbacks,
                                verbose=1)

            # save history metrics
            fold_train_acc.append(history.history.get('accuracy', [None])[-1])
            fold_train_loss.append(history.history.get('loss', [None])[-1])
            fold_val_acc.append(history.history.get('val_accuracy', [None])[-1])
            fold_val_loss.append(history.history.get('val_loss', [None])[-1])

            # Evaluate on test set
            test_res = model.evaluate(test_ds, steps=test_steps, verbose=0)
            test_loss, test_acc = test_res[0], test_res[1] if len(test_res) > 1 else (None, None)
            fold_test_acc.append(test_acc)
            fold_test_loss.append(test_loss)

            # Predictions for classification report
            y_pred_probs = model.predict(test_ds, steps=test_steps, verbose=0)
            y_pred_labels = np.argmax(y_pred_probs, axis=1)
            y_true_labels = test_labels_m  # numeric labels already

            report = classification_report(y_true_labels, y_pred_labels, target_names=all_classes, digits=4, zero_division=0)
            fold_class_reports.append(report)

            # Confusion matrix (use correct arrays)
            cm = confusion_matrix(y_true_labels, y_pred_labels)

            # Save confusion matrices with multiple color maps into the single PDF
            colormaps = {
                "blue": "Blues",
                "red": "Reds",
                "green": "Greens",
                "magenta": "magma",
                "viridis": "viridis"
            }
            for color_name, cmap_name in colormaps.items():
                plt.figure(figsize=(10, 8))
                sns.heatmap(cm, annot=True, fmt="d", cmap=cmap_name,
                            xticklabels=all_classes, yticklabels=all_classes)
                plt.xlabel("Predicted")
                plt.ylabel("True")
                plt.title(f"Fold {fold_no} - Confusion Matrix ({color_name})")
                plt.tight_layout()
                pdf.savefig()
                plt.close()

            # Accuracy & Loss plots
            plt.figure(figsize=(8, 4))
            plt.plot(history.history.get('accuracy', []), label="Train Acc")
            plt.plot(history.history.get('val_accuracy', []), label="Val Acc")
            plt.title(f"Fold {fold_no} Accuracy")
            plt.xlabel("Epoch")
            plt.ylabel("Accuracy")
            plt.legend()
            pdf.savefig()
            plt.close()

            plt.figure(figsize=(8, 4))
            plt.plot(history.history.get('loss', []), label="Train Loss")
            plt.plot(history.history.get('val_loss', []), label="Val Loss")
            plt.title(f"Fold {fold_no} Loss")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.legend()
            pdf.savefig()
            plt.close()

            # Classification report page
            fig, ax = plt.subplots(figsize=(8.5, 11))
            ax.axis("off")
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ax.text(0, 1, f"Classification Report - Fold {fold_no}\nGenerated: {timestamp}\n\n{report}",
                    fontsize=9, family="monospace", va="top")
            pdf.savefig(fig)
            plt.close(fig)

            print(f"Fold {fold_no} done. Test acc: {test_acc:.4f}, Test loss: {test_loss:.4f}")
            fold_no += 1

        # After all folds: summary page
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis('off')
        summary_text = "Fold | Train Acc | Train Loss | Val Acc | Val Loss | Test Acc | Test Loss\n"
        summary_text += "-"*90 + "\n"
        for i in range(len(fold_train_acc)):
            summary_text += (f"{i+1:>4} | "
                             f"{(fold_train_acc[i] if fold_train_acc[i] is not None else float('nan')):.4f} | "
                             f"{(fold_train_loss[i] if fold_train_loss[i] is not None else float('nan')):.4f} | "
                             f"{(fold_val_acc[i] if fold_val_acc[i] is not None else float('nan')):.4f} | "
                             f"{(fold_val_loss[i] if fold_val_loss[i] is not None else float('nan')):.4f} | "
                             f"{(fold_test_acc[i] if fold_test_acc[i] is not None else float('nan')):.4f} | "
                             f"{(fold_test_loss[i] if fold_test_loss[i] is not None else float('nan')):.4f}\n")
        ax.text(0, 1, "K-Fold Summary Metrics\n\n" + summary_text, fontsize=10, family="monospace", va="top")
        pdf.savefig(fig)
        plt.close(fig)

        # Aggregate stats page
        metrics = {
            "Train Acc": [v for v in fold_train_acc if v is not None],
            "Train Loss": [v for v in fold_train_loss if v is not None],
            "Val Acc": [v for v in fold_val_acc if v is not None],
            "Val Loss": [v for v in fold_val_loss if v is not None],
            "Test Acc": [v for v in fold_test_acc if v is not None],
            "Test Loss": [v for v in fold_test_loss if v is not None],
        }

        stats_text = "Metric        | Mean     | Std      | Min      | Max\n"
        stats_text += "-"*55 + "\n"
        for name, values in metrics.items():
            if len(values) == 0:
                stats_text += f"{name:<12} | {'nan':>7} | {'nan':>7} | {'nan':>7} | {'nan':>7}\n"
            else:
                stats_text += f"{name:<12} | {np.mean(values):.4f} | {np.std(values):.4f} | {np.min(values):.4f} | {np.max(values):.4f}\n"

        fig, ax = plt.subplots(figsize=(10, 6))
        ax.axis("off")
        ax.text(0, 1, "Aggregate K-Fold Statistics\n\n" + stats_text, fontsize=10, family="monospace", va="top")
        pdf.savefig(fig)
        plt.close(fig)

    print(f"\nK-Fold run complete. Report saved to: {PDF_REPORT}")

# -------------------------
# Run
# -------------------------
if __name__ == "__main__":
    run_kfold(BASE_PATH, num_folds=NUM_FOLDS, epochs=EPOCHS)


2025-10-02 09:33:24.083781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759397604.282564      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759397604.334219      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1759397619.296563      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Running on MirroredStrategy with GPUs: [LogicalDevice(name='/device:GPU:0', device_type='GPU')]
Number of replicas: 1
Detected classes: ['Amrapali', 'Banana', 'Bari 4', 'Fazli', 'GobindoBhog', 'GopalBhog', 'Harivanga', 'Himsagar', 'Khrishapat', 'Langra', 'RaniBhog', 'Sundari']

===== Fold 1 =====
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


E0000 00:00:1759397646.145503      19 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1759397649.583833      57 cuda_dnn.cc:529] Loaded cuDNN version 90300


153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6079 - loss: 1.3097
Epoch 1: val_loss improved from inf to 0.54573, saving model to best_model_fold1.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 395s 2s/step - accuracy: 0.6087 - loss: 1.3065 - val_accuracy: 0.8350 - val_loss: 0.5457 - learning_rate: 0.0010
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8780 - loss: 0.3485
Epoch 2: val_loss improved from 0.54573 to 0.16585, saving model to best_model_fold1.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 336s 2s/step - accuracy: 0.8781 - loss: 0.3484 - val_accuracy: 0.9453 - val_loss: 0.1659 - learning_rate: 0.0010
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9244 - loss: 0.2383
Epoch 3: val_loss did not improve from 0.16585
153/153 ━━━━━━━━━━━━━━━━━━━━ 333s 2s/step - accuracy: 0.9244 - loss: 0.2382 - val_accuracy: 0.9355 - val_loss: 0.1925 - learning_rate: 0.0010
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9417 - loss: 0.1689
Epoch 4: val_loss improved

E0000 00:00:1759401205.220938      19 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


...

153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6236 - loss: 1.2178
Epoch 1: val_loss improved from inf to 0.43461, saving model to best_model_fold3.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 360s 2s/step - accuracy: 0.6244 - loss: 1.2150 - val_accuracy: 0.8725 - val_loss: 0.4346 - learning_rate: 0.0010
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8903 - loss: 0.3504
Epoch 2: val_loss improved from 0.43461 to 0.24112, saving model to best_model_fold3.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 331s 2s/step - accuracy: 0.8904 - loss: 0.3502 - val_accuracy: 0.9232 - val_loss: 0.2411 - learning_rate: 0.0010
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9233 - loss: 0.2295
Epoch 3: val_loss improved from 0.24112 to 0.20629, saving model to best_model_fold3.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 357s 2s/step - accuracy: 0.9233 - loss: 0.2294 - val_accuracy: 0.9395 - val_loss: 0.2063 - learning_rate: 0.0010
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9434 

E0000 00:00:1759408200.837667      19 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6270 - loss: 1.1962
Epoch 1: val_loss improved from inf to 0.37355, saving model to best_model_fold4.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 378s 2s/step - accuracy: 0.6278 - loss: 1.1934 - val_accuracy: 0.8758 - val_loss: 0.3735 - learning_rate: 0.0010
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8995 - loss: 0.3133
Epoch 2: val_loss improved from 0.37355 to 0.21713, saving model to best_model_fold4.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 348s 2s/step - accuracy: 0.8995 - loss: 0.3133 - val_accuracy: 0.9346 - val_loss: 0.2171 - learning_rate: 0.0010
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9269 - loss: 0.2252
Epoch 3: val_loss improved from 0.21713 to 0.18119, saving model to best_model_fold4.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 341s 2s/step - accuracy: 0.9269 - loss: 0.2250 - val_accuracy: 0.9379 - val_loss: 0.1812 - learning_rate: 0.0010
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9437 

E0000 00:00:1759411729.622534      19 meta_optimizer.cc:966] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/efficientnetb0_1/block2b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6332 - loss: 1.1872
Epoch 1: val_loss improved from inf to 0.38789, saving model to best_model_fold5.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 361s 2s/step - accuracy: 0.6340 - loss: 1.1846 - val_accuracy: 0.8725 - val_loss: 0.3879 - learning_rate: 0.0010
Epoch 2/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8873 - loss: 0.3441
Epoch 2: val_loss improved from 0.38789 to 0.15126, saving model to best_model_fold5.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 338s 2s/step - accuracy: 0.8874 - loss: 0.3440 - val_accuracy: 0.9493 - val_loss: 0.1513 - learning_rate: 0.0010
Epoch 3/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9243 - loss: 0.2343
Epoch 3: val_loss improved from 0.15126 to 0.13185, saving model to best_model_fold5.h5
153/153 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - accuracy: 0.9243 - loss: 0.2343 - val_accuracy: 0.9542 - val_loss: 0.1319 - learning_rate: 0.0010
Epoch 4/10
153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9547 